In [2]:
!pip install -q -U anthropic yfinance rich

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.6/848.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

In [4]:
from anthropic import Anthropic
from rich import print
import re
import yfinance as yf

In [6]:
client = Anthropic(api_key = ANTHROPIC_API_KEY)
MODEL_NAME = "claude-3-opus-20240229"

In [7]:
stock_message = {
    "role": "user",
    "content": "Find the current price of Apple stock"
}

In [8]:
message = client.messages.create(model = MODEL_NAME,
                                 max_tokens = 1024,
                                 messages = [stock_message]
                                 ).content[0].text

print("##### Before Function Calling ####\n\n" + message)

##### Before Function Calling ####

To find the current price of Apple (AAPL) stock, you can use various financial websites or apps that provide 
real-time stock quotes. As of April 11, 2023, at 12:00 PM EST, the price of Apple stock is $162.03 per share. 
However, please note that stock prices can change rapidly during trading hours, so the price may be different by 
the time you check it.

Some popular financial websites to find stock quotes include:

1. Google Finance: Simply search for "AAPL" or "Apple stock" on Google.

2. Yahoo Finance: Visit https://finance.yahoo.com/ and search for "AAPL" or "Apple."

3. NASDAQ: Go to https://www.nasdaq.com/ and search for "AAPL" or "Apple Inc."

4. Bloomberg: Check https://www.bloomberg.com/quote/AAPL:US for the current Apple stock price.

Remember that stock prices are subject to change based on various factors such as market conditions, company 
performance, and global events.

In [9]:
# 1. Define the stock price finding function
def get_stock_price(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)
    hist = stock.history(period="1d")
    current_price = hist['Close'].iloc[0]
    return current_price

In [10]:
# 2. Construct Tool description
tool_description = """
<tool_description>
    <tool_name>get_stock_price</tool_name>
    <description>
        Function for finding the current price of a stock using its ticker symbol.
    </description>
    <parameters>
        <parameter>
            <name>ticker_symbol</name>
            <type>str</type>
            <description>Ticker symbol of the stock</description>
        </parameter>
    </parameters>
</tool_description>
"""

In [11]:
# 3. Ask Claude
system_prompt = f"""
In this environment you have access to a set of tools you can use to answer the
user's question.

You may call them like this:
<function_calls>
    <invoke>
        <tool_name>$TOOL_NAME</tool_name>
        <parameters>
            <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
            ...
        </parameters>
    </invoke>
</function_calls>

Here are the tools available:
<tools>{tool_description}</tools>
"""

In [12]:
function_calling_message = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1024,
    messages=[stock_message],
    system=system_prompt
).content[0].text

# print(function_calling_message)
function_calling_message

'<function_calls>\n    <invoke>\n        <tool_name>get_stock_price</tool_name>\n        <parameters>\n            <ticker_symbol>AAPL</ticker_symbol>\n        </parameters>\n    </invoke>\n</function_calls>'

In [13]:
# 4. Extract parameters from response & call the function
def extract_between_tags(tag, string, strip=False):
    ext_list = re.findall(f"<{tag}>(.+?)</{tag}>", string, re.DOTALL)
    return [e.strip() for e in ext_list] if strip else ext_list

In [14]:
function_params = {"ticker_symbol": extract_between_tags("ticker_symbol", function_calling_message)[0]}
function_name = extract_between_tags("tool_name", function_calling_message)[0]
names_to_functions = {
    'get_stock_price': get_stock_price,
}
price = names_to_functions[function_name](**function_params)

In [15]:
# Construct function results
function_results = f"""
<function_results>
  <result>
    <tool_name>get_stock_price</tool_name>
    <stdout>{price}</stdout>
  </result>
</function_results>"""

In [16]:
# 5. Send all messages back to Claude
partial_assistant_message = function_calling_message + function_results

In [17]:
final_message = client.messages.create(model = MODEL_NAME,
                                       max_tokens = 1024,
                                       messages=[stock_message,
                                        {
                                            "role": "assistant",
                                            "content": partial_assistant_message
                                        }
                                                 ],
                                       system=system_prompt
                                       ).content[0].text

In [19]:
print("\n\n##### After Function Calling #####"+ final_message)

##### After Function Calling #####

To find the current price of Apple stock, I used the get_stock_price tool, passing in 'AAPL' as the ticker symbol 
for Apple. The tool returned the current price of $169.0.